# ResNet50

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%run /content/drive/MyDrive/BINF_4008_Final_Project/Scripts/Metrics.ipynb
%run /content/drive/MyDrive/BINF_4008_Final_Project/Scripts/Models.ipynb
%run /content/drive/MyDrive/BINF_4008_Final_Project/Scripts/Trainer.ipynb
%run /content/drive/MyDrive/BINF_4008_Final_Project/Scripts/CustomDataset.ipynb

In [3]:
# import libraries
import pandas as pd
import numpy as np

import os

import pickle

import matplotlib.pyplot as plt
%matplotlib notebook

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import ResNet50_Weights
from torch.utils.data import Dataset, DataLoader

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error

In [4]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# load metadata
metadata = pd.read_csv('/content/drive/MyDrive/BINF_4008_Final_Project/Data/metadata.csv')

In [6]:
# split into different sets
train_images = metadata[metadata['split'] == 'train']['id'].astype(str)
train_tab = metadata[metadata['split'] == 'train']['male']
train_labels = metadata[metadata['split'] == 'train']['boneage']

val_images = metadata[metadata['split'] == 'val']['id'].astype(str)
val_tab = metadata[metadata['split'] == 'val']['male']
val_labels = metadata[metadata['split'] == 'val']['boneage']

test_images = metadata[metadata['split'] == 'test']['id'].astype(str)
test_tab = metadata[metadata['split'] == 'test']['male']
test_labels = metadata[metadata['split'] == 'test']['boneage']

In [7]:
test_metadata = metadata[metadata['split'] == 'test']

test_male_metadata = test_metadata[test_metadata['male'] == 1]
test_female_metadata = test_metadata[test_metadata['male'] == 0]

test_male_images = test_male_metadata['id'].astype(str)
test_male_tab = test_male_metadata['male']
test_male_labels = test_male_metadata['boneage']

test_female_images = test_female_metadata['id'].astype(str)
test_female_tab = test_female_metadata['male']
test_female_labels = test_female_metadata['boneage']

## Create Datasets and DataLoaders

In [8]:
BATCH_SIZE = 64
EPOCHS = 7

MEAN = [0.485]
STD = [0.229]

transform_1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels = 3),
    transforms.ToTensor(),
    transforms.Normalize(mean = MEAN * 3, std = STD * 3)
])
transform_2 = transforms.Compose([
    transforms.RandomAffine(degrees = (0, 90), translate = (0.1, 0.1), scale = None, shear = (0, 10)),
    transforms.RandomHorizontalFlip(0.3),
    transforms.RandomVerticalFlip(0.3)
])

In [9]:
train_dataset = CustomDataset('/content/drive/MyDrive/BINF_4008_Final_Project/Data/all_images/', train_images, train_labels, tabular_data = train_tab, transforms = [transform_1, transform_2])
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 2)

val_dataset = CustomDataset('/content/drive/MyDrive/BINF_4008_Final_Project/Data/all_images/', val_images, val_labels, tabular_data = val_tab, transforms = [transform_1])
val_dataloader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 2)

test_dataset = CustomDataset('/content/drive/MyDrive/BINF_4008_Final_Project/Data/all_images/', test_images, test_labels, tabular_data = test_tab, transforms = [transform_1])
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 2)

test_male_dataset = CustomDataset('/content/drive/MyDrive/BINF_4008_Final_Project/Data/all_images/', test_male_images, test_male_labels, tabular_data = test_male_tab, transforms = [transform_1])
test_male_dataloader = DataLoader(test_male_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 2)

test_female_dataset = CustomDataset('/content/drive/MyDrive/BINF_4008_Final_Project/Data/all_images/', test_female_images, test_female_labels, tabular_data = test_female_tab, transforms = [transform_1])
test_female_dataloader = DataLoader(test_female_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 2)

## Define Model

In [10]:
class ResNet50Regression(BaseModel):

  def __init__(self, weights, device, tabular_data = False, tabular_hidden = None, output_hidden = None):

    super(ResNet50Regression, self).__init__(device)

    if weights == 'random':
      self.resnet50 = models.resnet50(weights = None)
      self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, 1)
      self.resnet50 = self.resnet50.apply(self.xavier_init)

    else:
      self.resnet50 = models.resnet50(weights = weights)
      self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, 1)

    self.tabular_data = tabular_data

    if self.tabular_data:
      self.tabular_hidden = tabular_hidden
      self.output_hidden = output_hidden

    #   self.resnet50.classifier[-1] = nn.Linear(self.resnet50.classifier[-1].in_features, self.resnet50.classifier[-1].in_features)
      self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, self.resnet50.fc.in_features)

      self.tabular_net = nn.Sequential(
        nn.Linear(1, self.tabular_hidden),
        nn.ReLU()
      )

    #   self.fusion_dim = self.resnet50.classifier[-1].out_features + self.tabular_hidden
      self.fusion_dim = self.resnet50.fc.out_features + self.tabular_hidden

      self.output_net = nn.Sequential(
        nn.Linear(self.fusion_dim, self.output_hidden),
        nn.ReLU(),
        nn.Linear(self.output_hidden, self.output_hidden),
        nn.ReLU(),
        nn.Linear(self.output_hidden, 1)
      )

      self.tabular_net = self.tabular_net.apply(self.xavier_init)
      self.output_net = self.output_net.apply(self.xavier_init)

  def xavier_init(self, m):

    if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
        nn.init.xavier_uniform_(m.weight)

  def forward(self, image_input, tabular_input = None):

    if self.tabular_data:

      image_features = self.resnet50(image_input)
      image_features = image_features.view(image_features.size(0), -1)

      tabular_features = self.tabular_net(tabular_input)

      combined_features = torch.cat((image_features, tabular_features), dim = 1)

      final_output = self.output_net(combined_features)

      return final_output

    else:

      return self.resnet50(image_input)

## ImageNet Weight Initialization No Gender Model

In [11]:
# model = ResNet50Regression(weights = ResNet50_Weights.IMAGENET1K_V1, device = device).to(device)
with open('/content/drive/MyDrive/BINF_4008_Final_Project/Pickles/resnet50_imagenet_no_gender_model.pkl', 'rb') as f:
  model = pickle.load(f)

In [12]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

In [13]:
trainer_and_evaluator = BaseTrainer(model, criterion, optimizer, train_dataloader, val_dataloader, test_dataloader, test_male_dataloader, test_female_dataloader, EPOCHS, 'resnet50_imagenet_no_gender', device)

In [14]:
# training_results = trainer_and_evaluator.train_and_validate()

In [15]:
# training_results

In [17]:
test_results = trainer_and_evaluator.bootstrap_test_set()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
# test_results

## Xavier Weight Initialization No Gender Model

In [19]:
# model = ResNet50Regression(weights = 'random', device = device).to(device)
with open('/content/drive/MyDrive/BINF_4008_Final_Project/Pickles/resnet50_xavier_no_gender_model.pkl', 'rb') as f:
  model = pickle.load(f)

In [20]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

In [21]:
trainer_and_evaluator = BaseTrainer(model, criterion, optimizer, train_dataloader, val_dataloader, test_dataloader, test_male_dataloader, test_female_dataloader, EPOCHS, 'resnet50_xavier_no_gender', device)

In [22]:
# train_results = trainer_and_evaluator.train_and_validate()

In [23]:
# train_results

In [24]:
test_results = trainer_and_evaluator.bootstrap_test_set()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
# test_results

## ImageNet Weight Initialization With Gender Model

In [26]:
# model = ResNet50Regression(weights = ResNet50_Weights.IMAGENET1K_V1, device = device, tabular_data = True, tabular_hidden = 50, output_hidden = 1000).to(device)
with open('/content/drive/MyDrive/BINF_4008_Final_Project/Pickles/resnet50_imagenet_w_gender_model.pkl', 'rb') as f:
  model = pickle.load(f)

In [27]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

In [28]:
trainer_and_evaluator = BaseTrainer(model, criterion, optimizer, train_dataloader, val_dataloader, test_dataloader, test_male_dataloader, test_female_dataloader, EPOCHS, 'resnet50_imagenet_w_gender', device)

In [29]:
# train_results = trainer_and_evaluator.train_and_validate()

In [30]:
# train_results

In [31]:
test_results = trainer_and_evaluator.bootstrap_test_set()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [32]:
# test_results

## Xavier Weight Initialization With Gender Model

In [33]:
# model = ResNet50Regression(weights = 'random', device = device, tabular_data = True, tabular_hidden = 50, output_hidden = 1000).to(device)
with open('/content/drive/MyDrive/BINF_4008_Final_Project/Pickles/resnet50_xavier_w_gender_model.pkl', 'rb') as f:
  model = pickle.load(f)

In [34]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

In [35]:
trainer_and_evaluator = BaseTrainer(model, criterion, optimizer, train_dataloader, val_dataloader, test_dataloader, test_male_dataloader, test_female_dataloader, EPOCHS, 'resnet50_xavier_w_gender', device)

In [36]:
# train_results = trainer_and_evaluator.train_and_validate()

In [37]:
# train_results

In [38]:
test_results = trainer_and_evaluator.bootstrap_test_set()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
# test_results

## Xavier Weight Initialization No Gender Model (10 Epochs)

In [40]:
# model = ResNet50Regression(weights = 'random', device = device).to(device)
with open('/content/drive/MyDrive/BINF_4008_Final_Project/Pickles/resnet50_xavier_no_gender_10_model.pkl', 'rb') as f:
  model = pickle.load(f)

In [41]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

In [42]:
trainer_and_evaluator = BaseTrainer(model, criterion, optimizer, train_dataloader, val_dataloader, test_dataloader, test_male_dataloader, test_female_dataloader, 10, 'resnet50_xavier_no_gender_10', device)

In [43]:
# train_results = trainer_and_evaluator.train_and_validate()

In [44]:
# train_results

In [45]:
test_results = trainer_and_evaluator.bootstrap_test_set()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
# test_results

## Xavier Weight Initialization With Gender Model (10 Epochs)

In [47]:
# model = ResNet50Regression(weights = 'random', device = device, tabular_data = True, tabular_hidden = 50, output_hidden = 1000).to(device)
with open('/content/drive/MyDrive/BINF_4008_Final_Project/Pickles/resnet50_xavier_w_gender_10_model.pkl', 'rb') as f:
  model = pickle.load(f)

In [48]:
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters())

In [49]:
trainer_and_evaluator = BaseTrainer(model, criterion, optimizer, train_dataloader, val_dataloader, test_dataloader, test_male_dataloader, test_female_dataloader, 10, 'resnet50_xavier_w_gender_10', device)

In [50]:
# train_results = trainer_and_evaluator.train_and_validate()

In [51]:
# train_results

In [52]:
test_results = trainer_and_evaluator.bootstrap_test_set()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
# test_results